# Infomap

Multi-level network clustering based on the [Map equation](http://www.mapequation.org/publications.html#Rosvall-Axelsson-Bergstrom-2009-Map-equation).

### The Map Equation

\begin{equation*}
L(M) = q*\curvearrowright H(\mathcal{Q}) + \sum*{i = 1}^{m}{p\_{\circlearrowright}^i H(\mathcal{P}^i)}
\end{equation*}

$L(M)$ measures the amount of information it takes to describe a random walk on a network given a partition of the network into modules $M$. It is a sum of the amount of information needed to describe the movements _between_ and _within_ the modules, which balances the goodness of fit with the complexity of the model. For more information, see [www.mapequation.org](http://www.mapequation.org).


### Import Infomap

Infomap includes an `examples/python` folder with some examples, including this notebook. Run `make` in that directory to build the python interface to a local folder.

The `infomap` package exposes two classes, `Infomap` and `MemInfomap`, that wraps an input `network`, an output `tree`, and a `run` method to run Infomap on the input network. The classes takes a string of [options](http://www.mapequation.org/code.html#Options) as input.


In [7]:
import infomap

## Simple example


In [2]:
infomapWrapper = infomap.Infomap("--two-level")

# Add link weight as an optional third argument
infomapWrapper.addLink(0, 1)
infomapWrapper.addLink(0, 2)
infomapWrapper.addLink(0, 3)
infomapWrapper.addLink(1, 0)
infomapWrapper.addLink(1, 2)
infomapWrapper.addLink(2, 1)
infomapWrapper.addLink(2, 0)
infomapWrapper.addLink(3, 0)
infomapWrapper.addLink(3, 4)
infomapWrapper.addLink(3, 5)
infomapWrapper.addLink(4, 3)
infomapWrapper.addLink(4, 5)
infomapWrapper.addLink(5, 4)
infomapWrapper.addLink(5, 3)

infomapWrapper.run()

tree = infomapWrapper.tree

print(
    "Found %d modules with codelength: %f" % (tree.numTopModules(), tree.codelength())
)

print("\n#node module")
for node in tree.leafIter():
    print("%d %d" % (node.physIndex, node.moduleIndex()))

Found 2 modules with codelength: 2.320730

#node module
0 0
1 0
2 0
3 1
4 1
5 1


## Simple example with file


In [3]:
name = "ninetriangles"
filename = "../../{}.net".format(name)

infomapWrapper = infomap.Infomap("")

infomapWrapper.readInputData(filename)

infomapWrapper.run()

tree = infomapWrapper.tree

print("Found %d levels with codelength: %f" % (tree.maxDepth(), tree.codelength()))
print("Hierarchical solution:\n#path flow node")
for node in tree.treeIter():
    if node.isLeaf:
        print(
            '%s %f "%s"'
            % (
                ":".join(map(lambda n: str(n + 1), node.path())),
                node.data.flow,
                node.data.name,
            )
        )

Found 3 levels with codelength: 3.462273
Hierarchical solution:
#path flow node
1:1:1 0.038462 "4"
1:1:2 0.038462 "5"
1:1:3 0.038462 "6"
1:2:1 0.038462 "7"
1:2:2 0.038462 "8"
1:2:3 0.038462 "9"
1:3:1 0.038462 "1"
1:3:2 0.038462 "3"
1:3:3 0.025641 "2"
2:1:1 0.038462 "19"
2:1:2 0.038462 "20"
2:1:3 0.038462 "21"
2:2:1 0.038462 "22"
2:2:2 0.038462 "23"
2:2:3 0.038462 "24"
2:3:1 0.038462 "25"
2:3:2 0.038462 "26"
2:3:3 0.025641 "27"
3:1 0.038462 "10"
3:2 0.038462 "11"
3:3 0.038462 "12"
4:1 0.038462 "16"
4:2 0.038462 "17"
4:3 0.038462 "18"
5:1 0.038462 "13"
5:2 0.038462 "15"
5:3 0.025641 "14"


## Memory networks

With memory networks, the flow between two nodes depends on how you arrived at the first node. This higher-order relationships can be described by trigrams as in the example below:


### Trigrams


In [4]:
infomapWrapper = infomap.MemInfomap("--two-level")

# Trigrams represents a path from node A through B to C.
# Add link weight as an optional fourth argument
infomapWrapper.addTrigram(0, 2, 0)
infomapWrapper.addTrigram(0, 2, 1)
infomapWrapper.addTrigram(1, 2, 1)
infomapWrapper.addTrigram(1, 2, 0)
infomapWrapper.addTrigram(1, 2, 3)
infomapWrapper.addTrigram(3, 2, 3)
infomapWrapper.addTrigram(2, 3, 4)
infomapWrapper.addTrigram(3, 2, 4)
infomapWrapper.addTrigram(4, 2, 4)
infomapWrapper.addTrigram(4, 2, 3)
infomapWrapper.addTrigram(4, 3, 3)

infomapWrapper.run()

tree = infomapWrapper.tree

print(
    "Found %d modules with codelength: %f" % (tree.numTopModules(), tree.codelength())
)

print("\n#node module")
for node in tree.leafIter():
    print("%d %d" % (node.physIndex, node.moduleIndex()))

Found 3 modules with codelength: 1.227078

#node module
4 0
3 0
2 0
1 1
2 1
0 2


#### Overlapping modules

Notice that node `2` in the example below exists in both module `0` and `1`. This is because `MemInfomap` partitions the higher-order state network which can include multiple state nodes for each physical node. For trigrams, a state node is a pair of `previousNode node`. To keep the state network in the output tree, add the `--expanded` flag to configure `MemInfomap`:


In [5]:
# Store expanded state network
infomapWrapper = infomap.MemInfomap("--two-level --expanded")

infomapWrapper.addTrigram(0, 2, 0)
infomapWrapper.addTrigram(0, 2, 1)
infomapWrapper.addTrigram(1, 2, 1)
infomapWrapper.addTrigram(1, 2, 0)
infomapWrapper.addTrigram(1, 2, 3)
infomapWrapper.addTrigram(3, 2, 3)
infomapWrapper.addTrigram(2, 3, 4)
infomapWrapper.addTrigram(3, 2, 4)
infomapWrapper.addTrigram(4, 2, 4)
infomapWrapper.addTrigram(4, 2, 3)
infomapWrapper.addTrigram(4, 3, 3)

infomapWrapper.run()

tree = infomapWrapper.tree

print(
    "Found %d modules with codelength: %f" % (tree.numTopModules(), tree.codelength())
)

print("\n#previousNode node module")
for node in tree.leafIter():
    print("%d %d %d" % (node.stateIndex, node.physIndex, node.moduleIndex()))

Found 3 modules with codelength: 1.227078

#previousNode node module
2 3 0
3 4 0
2 4 0
3 2 0
4 2 0
2 1 1
0 2 1
1 2 1
2 0 2


As seen in the expanded output above, node `2` is represented by four state nodes partitioned into two modules depending on where you come from; if you go to node `2` from node `0` or `1` you are still considered to be in module `1`, but if you go to node `2` from node `3` and `4` you are still considered to be in module `0`.


### Multi-layer networks


In [6]:
infomapWrapper = infomap.MemInfomap("--two-level --expanded")

# from (layer, node) to (layer, node) weight
infomapWrapper.addMultiplexLink(2, 1, 1, 2, 1.0)
infomapWrapper.addMultiplexLink(1, 2, 2, 1, 1.0)
infomapWrapper.addMultiplexLink(3, 2, 2, 3, 1.0)

infomapWrapper.run()

tree = infomapWrapper.tree

print(
    "Found %d modules with codelength: %f" % (tree.numTopModules(), tree.codelength())
)

print("\n#layer node module:")
for node in tree.leafIter():
    print("%d %d %d" % (node.stateIndex, node.physIndex, node.moduleIndex()))

Found 3 modules with codelength: 0.865437

#layer node module:
2 1 0
1 2 0
2 3 1
3 2 1
0 0 2


### General state networks

The [state format](http://www.mapequation.org/code.html#State-format) describes the exact network used internally by Infomap. It can model both ordinary networks and memory networks (of variable order).


In [7]:
import tempfile

stateNetwork = """
*Vertices 4
1 "PRE"
2 "SCIENCE"
3 "PRL"
4 "BIO"
# *ngrams
# 1 2 3
# 1 2 2 3
# 4 2 4
*States
1 2 "1 2"
2 3 "2 3"
3 2 "1 2 2"
4 2 "4 2"
5 4 "2 4"
*Links
1 2
3 2
4 5
"""

filename = "%s/stateNetwork.net" % tempfile.gettempdir()
with open(filename, "w") as fp:
    fp.write(stateNetwork)
print("Wrote state network to file '{}'".format(filename))

Wrote state network to file '/var/folders/cm/8zsqg6xn2911z_jpkcxyxb4w0000gn/T/stateNetwork.net'


In [8]:
infomapWrapper = infomap.MemInfomap("-i states --expanded")

infomapWrapper.readInputData(filename)

infomapWrapper.run()

tree = infomapWrapper.tree

print(
    "Found %d modules with codelength: %f" % (tree.numTopModules(), tree.codelength())
)

print("State-expanded clusters:\n#stateIndex physicalIndex module name:")
for node in tree.leafIter():
    print(
        '%d %d %d "%s"'
        % (node.stateIndex, node.physIndex, node.moduleIndex(), node.data.name)
    )

Found 2 modules with codelength: 0.000000
State-expanded clusters:
#stateIndex physicalIndex module name:
1 2 0 "2 PRL"
0 1 0 "1 SCIENCE"
2 1 0 "3 SCIENCE"
4 3 1 "5 BIO"
3 1 1 "4 SCIENCE"


## Infomap + NetworkX

Generate and draw a network with NetworkX, colored
according to the community structure found by Infomap.


In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.colors as colors
%matplotlib inline

In [12]:
def findCommunities(G):
    """
    Partition network with the Infomap algorithm.
    Annotates nodes with 'community' id and return number of communities found.
    """
    infomapWrapper = infomap.Infomap("--two-level --silent")

    print("Building Infomap network from a NetworkX graph...")
    for e in G.edges():
        infomapWrapper.addLink(*e)

    print("Find communities with Infomap...")
    infomapWrapper.run()

    tree = infomapWrapper.tree

    communities = {}
    for node in tree.leafIter():
        communities[node.originalLeafIndex] = node.moduleIndex()

    nx.set_node_attributes(G, name="community", values=communities)
    return tree.numTopModules()

In [13]:
def drawNetwork(G):
    # position map
    pos = nx.spring_layout(G)
    # community ids
    communities = [v for k, v in nx.get_node_attributes(G, "community").items()]
    numCommunities = max(communities) + 1
    # color map from http://colorbrewer2.org/
    cmapLight = colors.ListedColormap(
        ["#a6cee3", "#b2df8a", "#fb9a99", "#fdbf6f", "#cab2d6"],
        "indexed",
        numCommunities,
    )
    cmapDark = colors.ListedColormap(
        ["#1f78b4", "#33a02c", "#e31a1c", "#ff7f00", "#6a3d9a"],
        "indexed",
        numCommunities,
    )

    # Draw edges
    nx.draw_networkx_edges(G, pos)

    # Draw nodes
    nodeCollection = nx.draw_networkx_nodes(
        G, pos=pos, node_color=communities, cmap=cmapLight
    )
    # Set node border color to the darker shade
    darkColors = [cmapDark(v) for v in communities]
    nodeCollection.set_edgecolor(darkColors)

    # Draw node labels
    for n in G.nodes():
        plt.annotate(
            n,
            xy=pos[n],
            textcoords="offset points",
            horizontalalignment="center",
            verticalalignment="center",
            xytext=[0, 0],
            color=cmapDark(communities[n]),
        )

    plt.axis("off")
    # plt.savefig("karate.png")
    plt.show()

In [14]:
G = nx.karate_club_graph()

findCommunities(G)

drawNetwork(G)

Building Infomap network from a NetworkX graph...
Find communities with Infomap...


AttributeError: 'InfoNode' object has no attribute 'leafIter'